In [1]:
#Step 1: We have import the pretrained model or load the mobilenet model
from keras.applications import resnet50
from keras.layers import GlobalAveragePooling2D
import cv2

model = resnet50.ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False)

model.layers[0].trainable
model.summary()
#Step2: Freeze all layers of the model expext the last layers as we have to make
#changes in that layer
for layers in model.layers:
    layers.trainable=False

model.layers[1].output
#Now add the layer which we have created to the ResNet50 layers
#where we have freezed all the layers

from keras.models import Sequential

from keras.layers import Dense

top_model = model.output
top_model

top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(2048,activation='relu')(top_model)
top_model = Dense(1500,activation='relu')(top_model)
top_model = Dense(1200,activation='relu')(top_model)
top_model = Dense(700,activation='relu')(top_model)
top_model = Dense(4,activation='softmax')(top_model)



from keras.models import Model
newmodel=Model(inputs=model.input ,outputs=top_model)

# Now compile the model and we have use imagedatagenerator to generate our images for the training

from keras.optimizers import RMSprop

newmodel.compile(optimizer = RMSprop(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])


import numpy as np

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'face_data/face_data/Train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        'face_data/face_data/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
# Now finally train the model and after the epochs completed then save that model
newmodel.fit(
        training_set,
        steps_per_epoch=200,
        epochs=1,
        validation_data=test_set,
        validation_steps=800)

Using TensorFlow backend.
C:\Users\RITIK\AppData\Local\conda\conda\envs\mytensor\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

Found 49 images belonging to 4 classes.
Found 9 images belonging to 4 classes.
Epoch 1/1
200/200 [==============================] - 3976s 20s/step - loss: 1.2877 - accuracy: 0.8880 - val_loss: 2.5217 - val_accuracy: 0.2222


In [ ]:
#Now we save  the model .

newmodel.save("newmodel_resnet.h5")

In [ ]:
#Now load the saved model to test our model.
from keras.models import load_model

classifier = load_model('newmodel_resnet.h5')

In [ ]:
#Test our model with the testing images we have putted in the folder.
#Here we have made a small python programm to do so and used cv2 module to show the images.
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_rec = {"[0]": "akash_agarwal", 
                      "[1]": "palak",
                      "[2]": "saurav_patel",
                      "[3]": "Shyamnedra_Sengar"}

face_rec_n = {"n0": "akash_agarwal", 
                      "n1": "palak",
                      "n2": "saurav_patel",
                      "n3": "Shyamendra_Sengar"}

def draw_test(name, pred, im):
    face = face_rec[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_rec_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,4):
    input_im = getRandomImage("face_data/face_data/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original,(224,224), None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()